<a href="https://colab.research.google.com/github/wherzberg/CNN-Introduction/blob/main/Smiley_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smiley Image Classifier
 - Created by Billy Herzberg
 - william.herzberg@marquette.edu

This notebook will generate simulated images that do or do not have smiley faces and use a convolutional neural network to classify them.  

# Import Libraries and Define Functions

In [ ]:
import tensorflow as tf
print("Using Tensorflow version",tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib        as mpl                 # There are a few other things needed from matplotlib

# Also set some things for plotting
plt.style.use('seaborn-whitegrid')
plt.rcParams["axes.grid"] = False

In [ ]:
def genData(N,n,m,p,A,B):
    # This function will generate a data set of blurry smiley face images
    # Inputs:
    # -> N = Number of samples
    # -> n = height of the images
    # -> m = width of the images
    # -> p = probability of a smiley face
    # -> A = magnitude of the smiley feature
    # -> B = standard deviation of the blurry noise
    # Outputs:
    # -> data = dictionary with X and Y
    #       X = a [N,n,m,1] array of image samples
    #       Y = a [N] array of image classifiers (1 for smiley 0 for nothing)

    # Start by building the blurry backgrounds for X and a vector of 0's for Y
    X = np.random.normal(loc=0,scale=B,size=[N,n,m,1])
    Y = np.zeros(shape=[N])

    # Then loop through each sample and maybe put a smiley
    for k in range(N):

        r = np.random.uniform()
        if r < p: # We need to add a smiley
            
            # Find the top left corner of the smiley
            row = np.random.randint(low=1,high=(n-3))
            col = np.random.randint(low=1,high=(m-3))

            # Then increase the values at the eyes and mouth
            X[k,row  ,col  ,0] += A
            X[k,row  ,col+2,0] += A
            X[k,row+2,col  ,0] += A
            X[k,row+2,col+1,0] += A
            X[k,row+2,col+2,0] += A

            # Also place a 1 in the Y vector
            Y[k] = 1

    # Now prepare the output as a dictionary and return
    data = {
        'X': X,
        'Y': Y
    }
    return data

print("genData is defined")

In [ ]:
def displaySample(X,Y,k,A,B):
    # This function will display one sample image with a title of either smiley 
    # or no smiley.
    # Inputs:
    # -> X = a [N,n,m] array of image samples
    # -> Y = a [N] array of image classifiers (1 for smiley 0 for nothing)
    # -> k = the sample number to display

    # Pick out the sample
    x = X[k,:,:].squeeze()

    # Decide on a title
    if Y[k] == 1:
        t = "Sample " + str(k) + ": Smiley"
    else:
        t = "Sample " + str(k) + ": No Smiley"
    
    #
    # Set the norm and colormap
    norm = mpl.colors.Normalize(vmin=-3*B, vmax=3*B+A)
    cmap='cool'

    # Display the image
    plt.imshow(x, norm=norm, cmap=cmap)
    plt.title(t)
    plt.axis('off')
    plt.colorbar( plt.cm.ScalarMappable(norm=norm, cmap=cmap) )
    plt.show()

print("displaySample is defined")

In [ ]:
def displayIntermediateOutput(model,layer,X,Y,k):
    # This function will display output from a layer in the model for sample k 
    # from set X.
    # Inputs:
    # -> model = the model to take intermediate output from
    # -> layer = the layer number to take output from
    # ->     X = data size [N,n,m,1] that can be fed into the network
    # ->     Y = data size [N] that has 1 if there is a smiley, else 0
    # ->     k = index of the data sample to be plotted

    # Pick out the sample, keeping dimensions
    x = X[k,:,:,:].reshape(1,X.shape[1],X.shape[2],1)
    y = Y[k]

    # snip the model so the output is at the desired layer
    model_snip = tf.keras.Model( inputs=model.input, outputs=model.layers[layer].output )

    # Send the sample through both models
    #ypred1 = model.predict(x).squeeze()  # This works but gives a warning so below is used instead. I don't get it
    #ypred2 = model_snip.predict(x).squeeze()  # This works but gives a warning so below is used instead. I don't get it
    ypred1 = np.array(model.predict_step(x)).squeeze()
    ypred2 = np.array(model_snip.predict_step(x)).squeeze()

    # Make the title
    if y == 1:
        t = "Sample " + str(k) + ": Smiley, YPred="    + str(ypred1)
    else:
        t = "Sample " + str(k) + ": No Smiley, YPred=" + str(ypred1)
    
    # Set the norm and colormap
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    cmap='cool'

    # Display the image
    plt.imshow(ypred2, norm=norm, cmap=cmap)
    plt.title(t)
    plt.axis('off')
    plt.colorbar( plt.cm.ScalarMappable(norm=norm, cmap=cmap) )
    plt.show()

print("displayIntermediateOutput is defined")

In [ ]:
def displayFilterChannel(model,layer,index1,index2):
    # Display a filter from a convolutional layer
    # Inputs:
    # -> model  = the model to take intermediate output from
    # -> layer  = the layer number to take output from
    # -> index1 = the filter channel (what channel of the input)
    # -> index2 = the filter index (which filter to look at)

    # Get the weights from the layer
    W = model.layers[layer].get_weights()[0]

    # Get the specific filter channel
    w = W[:,:,index1,index2].squeeze()

    # Make a title
    t = "Kernel Values"

    # Display the image
    plt.imshow(w)
    plt.title(t)
    plt.axis('off')
    plt.colorbar()
    plt.show()

print("displayFilterChannel is defined")

# Simulate a Data Set

In [ ]:
# Make a data set
N = 100    # Number of samples
n = 10     # Height
m = 10     # Width
p = 0.5    # Probability of a smiley
A = 5      # Intensity of smiley
B = 1      # Standard deviation of normal noise
data = genData(N,n,m,p,A,B)
X = data['X']    # Input images
Y = data['Y']    # Truths

# Display shapes
print("The shape of X:", data['X'].shape)
print("The shape of Y:", data['Y'].shape)

In [ ]:
for k in [0,1,2,3]:
    displaySample(X,Y,k,A,B)

In [ ]:
# Split into training and testing
train_size = 80
Xtrain = X[           :train_size, :, :, : ]
Xtest  = X[ train_size:          , :, :, : ]
Ytrain = Y[           :train_size ]
Ytest  = Y[ train_size:           ]

# Display shapes
print("The shape of Xtrain:", Xtrain.shape)
print("The shape of Xtest: ",  Xtest.shape)
print("The shape of Ytrain:", Ytrain.shape)
print("The shape of Ytest: ",  Ytest.shape)

# Build a CNN Model

In [ ]:
# Input Layer
X0 = tf.keras.Input( shape=( n,m,1 ) )

# Convolutional Layer
C1 = tf.keras.layers.Conv2D(
    filters     = 1,
    kernel_size = (5,5),
    strides     = (1,1),
    padding     = 'same',
    activation  = 'sigmoid'  # Sigmoid is usually the final activation function for classification
)(X0)

# Max Pool Layer
P1 = tf.keras.layers.MaxPool2D(
    pool_size    = (n,m)
)(C1)

# Flatten Layer
X99 = tf.keras.layers.Flatten()(P1)

# Put it all together
model = tf.keras.Model(inputs=X0, outputs=X99)
model.summary()

In [ ]:
# Compile
model.compile(
    optimizer = tf.keras.optimizers.SGD( learning_rate=0.01 ), # Gradient descent
    loss      = tf.keras.losses.BinaryCrossentropy(),
    metrics   = ['accuracy']
)
print("Model Compiled")

In [ ]:
# Display an image of the kernel
l = 1
ind1 = 0  # channel index
ind2 = 0  # filter number
displayFilterChannel(model,l,ind1,ind2)

In [ ]:
# Look at intermediate output
l = 1  # This is the convolutional layer
for k in [0,1,2,3]:
    displaySample(Xtest,Ytest,k,A,B)
    displayIntermediateOutput(model,l,Xtest,Ytest,k)

In [ ]:
# Predict
YtestPred = np.array(model.predict_step(Xtest)).squeeze()
print("Untrained Predictions")
print("   Sample      Truth       Prediction")
print("  ------------------------------------")
print(np.transpose(np.array([np.arange(N-train_size),Ytest,YtestPred])))

# Train the Model

In [ ]:
hist = model.fit(
    x = Xtrain,
    y = Ytrain,
    validation_data = (Xtest, Ytest),
    batch_size = 5,
    epochs = 50
)

# Evaluate the Model

In [ ]:
# Display an image of the kernel
l = 1
ind1 = 0  # channel index
ind2 = 0  # filter number
displayFilterChannel(model,l,ind1,ind2)    # -> model.get_weights()

In [ ]:
# Look at intermediate output
l = 1  # This is the convolutional layer
for k in [3,4,5,6]:
    displaySample(Xtest,Ytest,k,A,B)
    displayIntermediateOutput(model,l,Xtest,Ytest,k)

In [ ]:
YtestPred = np.array(model.predict_step(Xtest)).squeeze()
print("Trained Predictions")
print("   Sample      Truth       Prediction")
print("  ------------------------------------")
print(np.transpose(np.array([np.arange(N-train_size),Ytest,YtestPred])))

# Build a Bigger CNN Model

In [ ]:
# Build a bigger CNN
X0 = tf.keras.Input( shape=( n,m,1 ) )

# Convolution
C1 = tf.keras.layers.Conv2D(
    filters     = 30,
    kernel_size = (3,3),
    strides     = (1,1),
    padding     = 'same',
    activation  = 'relu'  
)(X0)

# Convolution
C2 = tf.keras.layers.Conv2D(
    filters     = 30,
    kernel_size = (3,3),
    strides     = (1,1),
    padding     = 'same',
    activation  = 'relu'  
)(C1)

# Pool
P1 = tf.keras.layers.MaxPool2D(
    pool_size    = (2,2),
    strides      = (2,2)
)(C2)

# Convolution
C3 = tf.keras.layers.Conv2D(
    filters     = 30,
    kernel_size = (3,3),
    strides     = (1,1),
    padding     = 'same',
    activation  = 'relu'  
)(P1)

# Convolution
C4 = tf.keras.layers.Conv2D(
    filters     = 1,
    kernel_size = (3,3),
    strides     = (1,1),
    padding     = 'same',
    activation  = 'sigmoid'  # Sigmoid is usually the final activation function for classification
)(C3)

# Pool
P2 = tf.keras.layers.MaxPool2D(
    pool_size    = (5,5)
)(C4)

X99 = tf.keras.layers.Flatten()(P2)

model2 = tf.keras.Model(inputs=X0, outputs=X99)
model2.summary()

In [ ]:
# Compile
model2.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
    loss      = tf.keras.losses.BinaryCrossentropy(),
    metrics   = ['accuracy']
)
print("Model Compiled")

In [ ]:
hist = model2.fit(
    x = Xtrain,
    y = Ytrain,
    validation_data = (Xtest, Ytest),
    batch_size = 5,
    epochs = 50
)

In [ ]:
# Display an image of the kernel
l = 2
ind1 = 0  # channel index
ind2 = 0  # filter number
for ind2 in [0,1,2,3,4,5]:
    displayFilterChannel(model2,l,ind1,ind2)

In [ ]:
YtestPred = np.array(model2.predict_step(Xtest)).squeeze()
print("Trained Predictions")
print("   Sample      Truth       Prediction")
print("  ------------------------------------")
print(np.transpose(np.array([np.arange(N-train_size),Ytest,YtestPred])))